In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import random
import cvxpy as cp
from scipy.io.arff import loadarff
from itertools import product, combinations, chain
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
import os
from joblib import Parallel, delayed
from sklearn.ensemble import RandomForestRegressor
import multiprocessing as mp
import pickle
from statistics import mean
from sklearn.model_selection import train_test_split

n_cores = mp.cpu_count()

In [2]:
## Read in ACS Income data. Preprocessing the same as in fig5.ipynb
X = pd.read_pickle("ACSIncome_transf_all.p")
states = pd.DataFrame(X['ST'].value_counts()).reset_index()['ST'][0:5]  # Use the first 5 states

def calc_Xbar(X_train, X_train_ST, subsamp, state):
    """Calculate Ehat[X]"""
    return (np.mean(X_train[X_train_ST == state].drop(["PINCP"], axis=1).sample(n=subsamp), axis=0))


def estimate_covariance(X_train, X_train_ST, Xbar_US, subsamp, states):
    """Estimate distributional covariance matrix"""
    Xbar_kp = Parallel(n_jobs=n_cores, verbose=0)(delayed(calc_Xbar)(X_train, X_train_ST, subsamp, t) for t in states)
    Xbar_kp_df = pd.DataFrame(Xbar_kp).T
    centered_Xbar = Xbar_kp_df.sub(Xbar_US.values, axis=0)
    distr_cov = centered_Xbar.cov()
    return distr_cov


Functions to calculate optimal samples and weights

In [3]:
def calc_penalized_weight(Sigma, N):
    """
    Calculate optimal weights from Eqn 2 of 3.1, Optimal sampling under size constraint
    
    :param Sigma: Estimated distributional covariance matrix
    :param N: Total sample budget
    :return: optimal weights
    """
    # Define the variable
    w = cp.Variable(Sigma.shape[0])

    # Define the objective 
    objective = cp.Minimize(cp.quad_form(w, Sigma) + (1 / N) * cp.sum(cp.abs(w)) ** 2)

    # Define the constraints (sum of weights equals 1)
    constraints = [cp.sum(w) == 1]

    # Solve convex optimization
    problem = cp.Problem(objective, constraints)
    problem.solve()

    # Extract the optimal weights
    w_optimal = w.value

    return w_optimal


def run_experiment_US(X, N, states):
    """
    Run experiment per Fig 4
    
    :param X: ACS Income data
    :param N: Total sample budget
    :param states: states to run experiment on
    :return: dict of results {mse from using individual states, mse from pooled data, mse from optimal sampling, optimal weights}
    """
    
    ## Test train split, stratified by state
    stratify_col = X['ST'].copy()
    X_train, X_test, X_train_ST, _ = train_test_split(X.drop(["ST"], axis=1), X['ST'], test_size=20000,
                                                      stratify=stratify_col)
    
    # Estimate distributional covariance matrix
    Xbar_US = np.mean(X_test.drop(["PINCP"], axis=1), axis=0)
    Sigma = estimate_covariance(X_train, X_train_ST, Xbar_US, 10000, states)
    
    # Calculate optimal weights
    opt_weights = calc_penalized_weight(Sigma, N)
    abs_opt_weights = np.abs(opt_weights)
    opt_n = np.round(abs_opt_weights / sum(abs_opt_weights) * N, 0)

    ##### comparisons
    sampled_states = [X_train[X_train_ST == s].sample(n=int(n)) for s, n in zip(states, opt_n)]
    pooled_N = round(N / len(states), 0)

    ## Prediction using pooled data
    pooled_states = pd.concat([X_train[X_train_ST == s].sample(n=int(pooled_N)) for s in states])
    y_hat_s = RandomForestRegressor().fit(pooled_states.drop(["PINCP"], axis=1), pooled_states["PINCP"]).predict(
        X_test.drop(["PINCP"], axis=1))
    mse_pooled = np.mean((X_test["PINCP"] - y_hat_s) ** 2)

    ## Prediction using individual state
    individual_states = [X_train[X_train_ST == s].sample(n=N) for s in states]
    y_hat_s_list = [RandomForestRegressor().fit(samples_s.drop(["PINCP"], axis=1), samples_s["PINCP"]).predict(
        X_test.drop(["PINCP"], axis=1)) for samples_s in individual_states]
    mse_individual_list = [np.mean((X_test["PINCP"] - y) ** 2) for y in y_hat_s_list]

    ## Prediction using optimal weighting
    y_hat_s_list = [RandomForestRegressor().fit(samples_s.drop(["PINCP"], axis=1), samples_s["PINCP"]).predict(
        X_test.drop(["PINCP"], axis=1)) for samples_s in sampled_states]
    y_hat_weighted_list = np.sum([w * y_hat_s for w, y_hat_s in zip(opt_weights, y_hat_s_list)], axis=0)
    weighted_mse = np.mean((X_test["PINCP"] - y_hat_weighted_list) ** 2)

    results = {
        'mse_individual_list': mse_individual_list,
        'mse_pooled': mse_pooled,
        'weighted_mse': weighted_mse,
        'opt_weights': opt_weights,
    }
    return results


Run experiments in parallel and wrangle for plotting.

In [ ]:
#### Caution, takes a while to run. 
#### The results are saved as 'us_opt_samples_agg_all.csv' and 'us_opt_samples_individual_all.csv' respectively
# # Samples
# n_k = 1000
# trials = 100
# N_seq = np.arange(50, 1100, 50)
# random.seed(1)
# 
# # Run experiments in parallel
# results_list = []
# for N in N_seq:
#     results = Parallel(n_jobs=n_cores, verbose=0)(
#         delayed(run_experiment_US)(X, N, states) for _ in range(trials))
#     results_list.append(results)


In [ ]:
# results_df_list = [pd.DataFrame({"state": states,
#                                  'weights': [np.mean([results_list[N][t].get('opt_weights')[s] for t in range(trials)])
#                                              for s in range(len(states))],
#                                  "mse_individual": [
#                                      np.mean([results_list[N][t].get('mse_individual_list')[s] for t in range(trials)])
#                                      for s in range(len(states))], }) for N in range(len(N_seq))
#                    ]
# weighted_mse = [np.mean([results_list[N][t].get('weighted_mse') for t in range(trials)]) for N in range(len(N_seq))]
# pooled_mse = [np.mean([results_list[N][t].get('mse_pooled') for t in range(trials)]) for N in range(len(N_seq))]
# 
# combined_data = pd.DataFrame()
# 
# for i, df in enumerate(results_df_list):
#     df['N_seq'] = N_seq[i]
#     combined_data = pd.concat([combined_data, df], axis=0)
# 
# melted_data = pd.melt(
#     combined_data,
#     id_vars=['N_seq', 'state'],
#     value_vars='mse_individual',
#     var_name='Metric',
#     value_name='MSE Value'
# )
# melted_data.to_csv("us_opt_samples_individual_all.csv", index=False)
# mse_data = pd.DataFrame({
#     'N_seq': N_seq,
#     'Weighted MSE': weighted_mse,
#     'Pooled MSE': pooled_mse,
# })
# mse_data.to_csv("us_opt_samples_agg_all.csv", index=False)
